# RAG embeddings

## 0. Libraries

In [1]:
import os, json, ast, requests
import numpy as np
import pandas as pd

# Optional fallback if faiss is not installed
try:
    import faiss
    _has_faiss = True
except Exception:
    _has_faiss = False
    from sklearn.neighbors import NearestNeighbors


## 1. Config

In [2]:
# ---------- CONFIG ----------
CSV_PATH = "/home/jggomez/Desktop/IRIS/iris-uc3m/data/Base de datos Noticias 27102025.xlsx - Noticias_embeddings.csv"
TEXT_COL = "textonoticia"
EMBED_COL = "embedding"   # column with JSON-list OR use emb_0... columns
METADATA_COLS = [
    "Género periodista",
    "Género personas noticias", "IA principal", "Significado IA",
    "Referencias políticas igualdad", "Denuncia desigualdad género",
    "Presencia mujeres racializadas", "Presencia  mujeres discapacidad",
    "Presencia diversidad generacional", "Menciona IA", "Lenguaje sexista",
    "Masculino genérico", "Hombre humanidad", "Uso dual zorra",
    "No uso cargos mujeres", "Sexismo social", "Androcentrismo_",
    "Mujeres sin nombre", "Asimetría mujer hombre", "Infantilización mujeres",
    "Denominación sexualizada", "Denominación redundante", "Denominación dependiente",
    "Criterios excepción", "Comparación mujeres/hombres"
]
OLLAMA_EMBED_URL = "http://localhost:11434/api/embed"
OLLAMA_MODEL = "embeddinggemma:300m"

## 2. Helpers

In [3]:
# === Helpers ===
def parse_embedding_cell(x):
    if isinstance(x, str):
        try:
            return json.loads(x)
        except Exception:
            try:
                return ast.literal_eval(x)
            except Exception:
                return None
    elif isinstance(x, (list, np.ndarray)):
        return list(x)
    else:
        return None

def l2_normalize(arr):
    arr = np.asarray(arr, dtype=np.float32)
    norms = np.linalg.norm(arr, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    return arr / norms

def get_ollama_embedding(text, url=OLLAMA_EMBED_URL, model=OLLAMA_MODEL, timeout=30):
    payload = {"model": model, "input": text}
    r = requests.post(url, json=payload, timeout=timeout)
    r.raise_for_status()
    data = r.json()
    # direct cases
    if isinstance(data, dict) and "embedding" in data and isinstance(data["embedding"], list) and len(data["embedding"])>0:
        return np.array(data["embedding"], dtype=np.float32)
    # recursive search for first numeric list
    def find_first_array(obj):
        if isinstance(obj, list):
            if obj and all(isinstance(x, (int, float)) for x in obj):
                return obj
            for el in obj:
                res = find_first_array(el)
                if res is not None:
                    return res
        if isinstance(obj, dict):
            for v in obj.values():
                res = find_first_array(v)
                if res is not None:
                    return res
        return None
    arr = find_first_array(data)
    if arr is None:
        raise RuntimeError(f"No embedding array found in Ollama response: {data}")
    if len(arr) == 0:
        raise RuntimeError("Embedding returned empty by Ollama.")
    return np.array(arr, dtype=np.float32)

## 3. Load data

In [4]:
# === 1) Load CSV and embeddings ===
df = pd.read_csv(CSV_PATH)
print("Loaded rows:", len(df))

# detect embedding format
if EMBED_COL in df.columns:
    df["__embedding_list"] = df[EMBED_COL].apply(parse_embedding_cell)
    df = df[df["__embedding_list"].notna()].reset_index(drop=True)
    embeddings = np.vstack(df["__embedding_list"].values).astype(np.float32)
else:
    emb_cols = [c for c in df.columns if c.startswith("emb_")]
    if not emb_cols:
        raise ValueError("No embeddings found: neither column 'embedding' nor emb_ cols.")
    emb_cols = sorted(emb_cols, key=lambda x: int(x.split("_")[1]))
    embeddings = df[emb_cols].to_numpy(dtype=np.float32)
    df["__embedding_list"] = list(embeddings.tolist())

n, d = embeddings.shape
print("Embeddings shape:", embeddings.shape)

# normalize
emb_norm = l2_normalize(embeddings)

Loaded rows: 1246
Embeddings shape: (1246, 768)


## 4. Build index

In [5]:
# === 2) Build index (FAISS if available, else sklearn) ===
if _has_faiss:
    index = faiss.IndexFlatIP(d)  # inner product on normalized vectors = cosine similarity
    index.add(emb_norm)
    use_faiss = True
    print("FAISS index built.")
else:
    nbrs = NearestNeighbors(n_neighbors=10, metric="cosine").fit(embeddings)  # cosine distance
    use_faiss = False
    print("Sklearn NearestNeighbors built (cosine).")

FAISS index built.


## 5. Retrieve

In [6]:
# === 3) Retrieval function ===
def retrieve_top_k(query_text, k=3, return_metadata_cols=None):
    if return_metadata_cols is None:
        return_metadata_cols = METADATA_COLS + [TEXT_COL]
    # 1) embed query
    q = get_ollama_embedding(query_text)
    if q.shape[0] != d:
        raise RuntimeError(f"Query embedding dim {q.shape[0]} != index dim {d}")
    q = q.reshape(1, -1).astype(np.float32)
    q_norm = l2_normalize(q)
    results = []
    if use_faiss:
        # FAISS returns inner-product (higher = more similar)
        scores, idxs = index.search(q_norm, k)
        scores = scores[0].tolist()
        idxs = idxs[0].tolist()
        for score, idx in zip(scores, idxs):
            if idx < 0: continue
            row = df.iloc[idx]
            meta = {c: (row[c] if c in row.index else None) for c in return_metadata_cols}
            results.append({
                "doc_index": int(idx),
                "score": float(score),   # cosine similarity
                "text": str(row.get(TEXT_COL, "")),
                "metadata": meta
            })
    else:
        # sklearn returns cosine distance (0 best). convert to similarity = 1 - distance
        distances, idxs = nbrs.kneighbors(q, n_neighbors=k)
        distances = distances[0].tolist()
        idxs = idxs[0].tolist()
        for dist, idx in zip(distances, idxs):
            sim = 1.0 - float(dist)
            row = df.iloc[idx]
            meta = {c: (row[c] if c in row.index else None) for c in return_metadata_cols}
            results.append({
                "doc_index": int(idx),
                "score": sim,
                "text": str(row.get(TEXT_COL, "")),
                "metadata": meta
            })
    return results

## 6. Test

In [7]:
noticias_test = [
    # 1. Política — invisibilización + minimización + fuentes masculinas
    """
    El Gobierno anunció esta mañana una reestructuración completa del Ministerio de Ciencia,
    un área que la nueva ministra, Ana Robles, “intentará gestionar con prudencia”, según palabras
    del presidente. Durante la presentación, el presidente destacó la “experiencia técnica” de los
    secretarios de Estado —todos ellos hombres— y atribuyó a ellos la responsabilidad de “marcar el rumbo
    científico de los próximos años”.

    Aunque Robles tiene más de veinte años de trayectoria en investigación internacional,
    en el acto se insistió en su “carácter dialogante” y “capacidad de escucha”, mencionando su vida familiar
    en varias ocasiones. De los doce análisis citados como respaldo técnico, once fueron elaborados por expertos
    varones; la única investigadora mencionada apareció como “colaboradora”.

    La cobertura mediática posterior subrayó la “imagen cercana” de la ministra y su “tono suave”, comparándola
    con su predecesor, “conocido por su firmeza”. En ningún artículo se destacó el historial científico de Robles ni
    su liderazgo en proyectos europeos, mientras que múltiples titulares destacaron la “solidez del equipo masculino
    que la asesorará”. Organizaciones feministas han criticado la presentación por reproducir un modelo de liderazgo
    centrado en figuras masculinas y relegar a la ministra a un papel más simbólico que técnico.
    """,

    # 2. Tecnología — estereotipos + infantilización + atribución parcial
    """
    La empresa DataForge presentó hoy su nuevo asistente de inteligencia artificial, desarrollado por un equipo
    de más de 70 ingenieros. En los materiales de prensa, la compañía destacó repetidamente el papel de “los jóvenes talentos”
    masculinos del departamento de algoritmia, mostrando fotos y citas extensas de los hombres del equipo.

    Entre las desarrolladoras figura Laura M., una ingeniera con diez años de experiencia en aprendizaje profundo,
    pero el comunicado de la empresa la menciona únicamente como “una de las chicas del equipo de diseño”.
    En un vídeo promocional, Laura aparece sonriendo y sosteniendo una tableta sin participar en explicaciones técnicas.
    En redes sociales, la empresa respondió a un comentario diciendo que “Laura aporta sensibilidad al producto”.

    El producto ha recibido elogios por su capacidad de análisis predictivo, pero ninguna de las notas difundidas por
    DataForge atribuye esas mejoras al subequipo liderado por Laura, responsable del modelo de series temporales.
    Varias ONG tecnológicas criticaron la comunicación por reproducir estereotipos de género y por reducir el aporte técnico
    de las desarrolladoras a roles decorativos.
    """,

    # 3. Deportes — sexualización + androcentrismo + énfasis físico
    """
    La liga de baloncesto femenina arrancó este fin de semana con un lleno total en varios estadios, pero la mayoría
    de los titulares se centraron en la apariencia de las jugadoras. Varios medios destacaron que las “estrellas del equipo”
    lucieron “atuendos llamativos” y “peinados impecables”, dedicando tan solo unas líneas a los resultados deportivos.

    En la rueda de prensa posterior al partido, a la base del equipo, Marta C., se le preguntó por “cómo consigue
    mantenerse en forma mientras concilia su vida personal”, mientras que al entrenador masculino se le solicitaron análisis
    tácticos detallados. En el mismo artículo, un periodista afirmó que la “belleza natural” de Marta es “uno de los grandes
    atractivos de la competición”.

    Pese a su actuación sobresaliente —28 puntos y 11 asistencias—, su rendimiento fue descrito como “una sorpresa agradable”
    mientras que el pívot masculino del equipo filial, con un rendimiento menor, fue calificado como “una promesa consolidada
    del baloncesto nacional”.
    """,

    # 4. Economía/empresa — condescendencia + dudas sobre autoridad femenina
    """
    La directiva de la empresa logística TransMar asumió ayer la presidencia tras la renuncia del anterior responsable.
    En su comunicado oficial, varios consejeros —todos hombres— afirmaron que confían en que la nueva presidenta, Clara Díez,
    “sabrá manejar las relaciones internas con mano suave” y destacaron su “sensibilidad natural para tratar con los empleados”.

    Aunque Díez ha dirigido durante seis años la expansión internacional de la compañía, la noticia fue tratada en algunos medios
    como “un desafío inesperado para la ejecutiva”, insinuando que podría necesitar “apoyo permanente del equipo senior”.
    Un editorial señalaba: “A Clara se la ve trabajadora y muy organizada, aunque quizá le falte presencia para negociar
    con clientes importantes”.

    En contraste, cuando el director anterior fue nombrado, los medios resaltaron su “ambición”, “liderazgo firme”
    y “visión estratégica”, sin mencionar nunca su vida personal ni su apariencia.
    """,

    # 5. Sociedad — culpabilización de víctimas + minimización
    """
    La policía informó hoy sobre una agresión ocurrida en un bar de la ciudad. Aunque el informe oficial describe el suceso
    como un ataque en grupo contra una mujer, varios titulares insinuaron que la víctima “había bebido más de la cuenta”
    y “deambulaba sola de madrugada”.

    En algunos artículos se sugiere que la mujer “provocó la situación” al “llamar la atención de los hombres”, pese a que
    la investigación preliminar no indica conducta inapropiada por su parte. Los artículos incluyen declaraciones extensas
    de los hombres implicados, que alegan “malentendidos” y “bromas que se salieron de control”, mientras que la declaración
    de la víctima aparece resumida en una sola frase.

    Las organizaciones feministas señalan que este tipo de tratamiento mediático reproduce patrones de culpabilización
    de las mujeres víctimas y minimiza la responsabilidad de los agresores, reforzando estereotipos profundamente arraigados.
    """
]

In [8]:
# ============================================
# 🔎 SUMMARY TABULAR POR VARIABLE EN TOP-K
# ============================================

def summarize_variables(top_results, metadata_cols, min_width=40):
    """
    top_results: salida de retrieve_top_k(...)
    metadata_cols: variables del dataset
    min_width: ancho mínimo por columna (default=40)
    """

     # Header: Variable | Doc 1 | Doc 2 | Doc 3
    headers = ["VARIABLE"] + [f"Doc {i+1}" for i in range(len(top_results))]

    # Calculamos un ancho grande
    col_widths = [max(len(h), min_width) for h in headers]

    print("\n" + "="* (sum(col_widths) + 3 * (len(headers)-1)))
    print("📊 RESUMEN DE VARIABLES POR DOCUMENTO (FILAS = VARIABLES)")
    print("="* (sum(col_widths) + 3 * (len(headers)-1)))

   

    

    def fmt_row(cols):
        return " | ".join(str(c).ljust(w) for c, w in zip(cols, col_widths))

    # Print header
    print(fmt_row(headers))
    print("-" * (sum(col_widths) + 3 * (len(headers)-1)))

    # Rows: una variable por fila
    for var in metadata_cols:
        row = [var]
        for r in top_results:
            val = r["metadata"].get(var, "")
            row.append("" if val is None else str(val))
        print(fmt_row(row))

    print("="*(sum(col_widths) + 3 * (len(headers)-1)) + "\n")

In [9]:
# === Example usage ===
top3 = retrieve_top_k(noticias_test[4], k=3)

# imprimir los resultados básicos
for i, r in enumerate(top3, 1):
    print(f"Rank {i}: idx={r['doc_index']}, score={r['score']:.4f}")
    print("Preview:", r['text'][:200].replace("\n", " "))
    print("-"*50)

# resumen por variables
summarize_variables(top3, METADATA_COLS)

Rank 1: idx=1235, score=0.5982
Preview: muchas son las ocasiones en las que ponemos las noticias y vemos que en diferentes partes del mundo se han producido tiroteos o han alertado de individuos armados con comportamientos agresivos que, en
--------------------------------------------------
Rank 2: idx=1111, score=0.5909
Preview: la guardia civil investiga en mallorca un caso de manipulación pornográfica de imágenes con inteligencia artificial que afecta a chicas menores de edad. los autores fueron igualmente varios menores de
--------------------------------------------------
Rank 3: idx=385, score=0.5899
Preview: nuevo varapalo de bruselas a meta, la empresa matriz de facebook. la comisión europea ha denunciado este lunes a la compañía por violar la ley de mercados digitales (dma por sus siglas en inglés) con 
--------------------------------------------------

📊 RESUMEN DE VARIABLES POR DOCUMENTO (FILAS = VARIABLES)
VARIABLE                                 | Doc 1                   

## 7. Agents

In [10]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import tool

# === Tool: Wikipedia (ES) ===
wiki = WikipediaAPIWrapper(lang="es", top_k_results=3, doc_content_chars_max=2000)

@tool("wikipedia_search")
def wikipedia_search(query: str) -> str:
    """Busca en Wikipedia (es) y devuelve un resumen de los mejores resultados."""
    try:
        result = wiki.run(query)
        return result if isinstance(result, str) else str(result)
    except Exception as e:
        return f"Error en Wikipedia: {e}"


In [11]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool

ddg_search = DuckDuckGoSearchRun()

@tool("duckduckgo_search")
def duck_search(query: str) -> str:
    """Busca en la web usando DuckDuckGo y devuelve un resumen breve."""
    try:
        return ddg_search.run(query)
    except Exception as e:
        return f"Error en DuckDuckGo: {e}"


In [12]:

from langchain.agents import initialize_agent, AgentType
from crewai import Agent, Task, Crew
from langchain_community.llms import Ollama

# Inicializamos el modelo local
ollama_model = Ollama(model="gemma3:4b", base_url="http://localhost:11434")


# === Agente con herramientas (incluye Wikipedia) ===
tools = [wikipedia_search, duck_search]
react_agent = initialize_agent(
    tools=tools,
    llm=ollama_model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Ejemplo de uso
query = "Busca en la web actual quién es Pedro Sánchez y resume en 3 frases."
react_result = react_agent.invoke({"input": query})
print(react_result["output"])


/tmp/ipykernel_2975019/2613129858.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  ollama_model = Ollama(model="gemma3:4b", base_url="http://localhost:11434")
/tmp/ipykernel_2975019/2613129858.py:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/d



> Entering new AgentExecutor chain...
I need to find out who Pedro Sánchez is and summarize him in 3 sentences. I will use a web search to do this.
Action: duckduckgo_search
Action Input: "Pedro Sánchez"
Observation: No good DuckDuckGo Search Result was found
Thought:

/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Okay, DuckDuckGo didn't provide a helpful result. Let's try Wikipedia to get a more detailed overview.
Action: wikipedia_search
Action Input: "Pedro Sánchez"
Observation: Page: Pedro Sánchez
Summary: Pedro Sánchez Pérez-Castejón (Madrid, 29 de febrero de 1972)​ es un político y economista español, actual presidente del Gobierno de España desde 2018 (7 años, 5 meses y 17 días). Es secretario general del Partido Socialista Obrero Español (PSOE) desde 2017, cargo que ya había desempeñado entre 2014 y 2016.​
Licenciado en Ciencias Económicas y Empresariales por el Real Centro Universitario María Cristina de El Escorial y doctor en Economía y Empresa por la Universidad Camilo José Cela, su carrera política la comenzó como concejal en el Ayuntamiento de Madrid, entre 2004 y 2009. Luego fue diputado en el Congreso en la ix y x legislaturas. En 2014 sucedió a Alfredo Pérez Rubalcaba al frente de la secretaría general del PSOE, y en 2015 y 2016, fue el candidato propuesto por su partido a la pr

/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Okay, DuckDuckGo didn't provide a helpful result. Let's try Wikipedia to get a more detailed overview.
Action: wikipedia_search
Action Input: "Pedro Sánchez"
Observation: Page: Pedro Sánchez
Summary: Pedro Sánchez Pérez-Castejón (Madrid, 29 de febrero de 1972)​ es un político y economista español, actual presidente del Gobierno de España desde 2018 (7 años, 5 meses y 17 días). Es secretario general del Partido Socialista Obrero Español (PSOE) desde 2017, cargo que ya había desempeñado entre 2014 y 2016.​
Licenciado en Ciencias Económicas y Empresariales por el Real Centro Universitario María Cristina de El Escorial y doctor en Economía y Empresa por la Universidad Camilo José Cela, su carrera política la comenzó como concejal en el Ayuntamiento de Madrid, entre 2004 y 2009. Luego fue diputado en el Congreso en la ix y x legislaturas. En 2014 sucedió a Alfredo Pérez Rubalcaba al frente de la secretaría general del PSOE, y en 2015 y 2016, fue el candidato propuesto por su partido a la pr

/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Thought:Okay, DuckDuckGo didn't provide a helpful result. Let's try Wikipedia to get a more detailed overview.
Action: wikipedia_search
Action Input: "Pedro Sánchez"
Observation: Page: Pedro Sánchez
Summary: Pedro Sánchez Pérez-Castejón (Madrid, 29 de febrero de 1972)​ es un político y economista español, actual presidente del Gobierno de España desde 2018 (7 años, 5 meses y 17 días). Es secretario general del Partido Socialista Obrero Español (PSOE) desde 2017, cargo que ya había desempeñado entre 2014 y 2016.​
Licenciado en Ciencias Económicas y Empresariales por el Real Centro Universitario María Cristina de El Escorial y doctor en Economía y Empresa por la Universidad Camilo José Cela, su carrera política la comenzó como concejal en el Ayuntamiento de Madrid, entre 2004 y 2009. Luego fue diputado en el Congreso en la ix y x legislaturas. En 2014 sucedió a Alfredo Pérez Rubalcaba al frente de la secretaría general del PSOE, y en 2015 y 2016, fue el candidato propuesto por su partido

/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: No good DuckDuckGo Search Result was found
Thought:Okay, DuckDuckGo didn't provide a helpful result. Let's try Wikipedia to get a more detailed overview.
Action: wikipedia_search
Action Input: "Pedro Sánchez"
Observation: Page: Pedro Sánchez
Summary: Pedro Sánchez Pérez-Castejón (Madrid, 29 de febrero de 1972)​ es un político y economista español, actual presidente del Gobierno de España desde 2018 (7 años, 5 meses y 17 días). Es secretario general del Partido Socialista Obrero Español (PSOE) desde 2017, cargo que ya había desempeñado entre 2014 y 2016.​
Licenciado en Ciencias Económicas y Empresariales por el Real Centro Universitario María Cristina de El Escorial y doctor en Economía y Empresa por la Universidad Camilo José Cela, su carrera política la comenzó como concejal en el Ayuntamiento de Madrid, entre 2004 y 2009. Luego fue diputado en el Congreso en la ix y x legislaturas. En 2014 sucedió a Alfredo Pérez Rubalcaba al frente de la secretaría general del PSOE, y 

## 8. Agente de Igualdad

In [13]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool
from typing import List, Dict
import datetime

ddg_search = DuckDuckGoSearchRun()

def format_results(results: List[Dict]) -> str:
    """
    Formatea resultados en texto corto para el LLM.
    results: lista de dicts (según salida de ddg_search.run)
    """
    lines = []
    for i, r in enumerate(results[:5], start=1):
        title = r.get("title") or ""
        snippet = r.get("body") or r.get("snippet") or ""
        url = r.get("href") or r.get("url") or ""
        lines.append(f"{i}. {title}\n   {snippet}\n   {url}")
    return "\n\n".join(lines)

@tool("duckduckgo_search_eq")
def duck_search_eq(query: str) -> str:
    """
    Búsqueda web especializada para agente de igualdad.
    Devuelve: fecha_consulta | resumen_snippets | urls
    """
    try:
        raw = ddg_search.run(query)  # normalmente devuelve texto; algunos wrappers devuelven estructura
        # Intentamos manejar ambos formatos: string o lista/dict
        if isinstance(raw, str):
            return f"[{datetime.date.today()}] Resultados (texto):\n\n{raw}"
        # si es lista/dict:
        formatted = format_results(raw if isinstance(raw, list) else raw.get("results", []))
        return f"[{datetime.date.today()}] Resultados (formateados):\n\n{formatted}"
    except Exception as e:
        return f"Error en DuckDuckGo: {e}"


In [14]:
from langchain.tools import tool
from langchain_community.llms import Ollama

ollama_model = Ollama(model="gemma3:4b", base_url="http://localhost:11434")

@tool("analizar_igualdad_web")
def analizar_igualdad_web(query: str) -> str:
    """
    Pipeline:
      1) Hace búsqueda especializada (duck_search_eq)
      2) Llama al LLM para:
         - Resumir hallazgos en 3-4 líneas
         - Evaluar si hay sesgo (SIN_SEXISMO, SESGO_SUTIL, SEXISMO_EXPLÍCITO)
         - Extraer 2 evidencias (texto corto + url)
         - Sugerir 2 acciones correctivas (para periodista/editor)
    """
    # 1) obtener contexto desde la tool duck
    context = duck_search_eq(query)
    # 2) prompt para el LLM
    prompt = f"""
Eres un/una experta en detección de sesgo de género. Tienes este contexto obtenido de la web:
{context}

Tareas (devuelve JSON estricto con las claves abajo):
1) resumen: Síntesis en 3 frases.
2) etiqueta: una de ["SIN_SEXISMO","SESGO_SUTIL","SEXISMO_EXPLÍCITO"].
3) evidencias: lista con 2 objetos {{"texto": "...", "url": "...", "porqué": "..."}}
4) recomendaciones: lista con 2 recomendaciones prácticas para el periodista/editor.

Devuelve SOLO JSON.
"""
    try:
        resp = ollama_model(prompt)
        # resp podría ser objeto; convertir a string
        return str(resp)
    except Exception as e:
        return f"Error en LLM: {e}"


In [15]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

equality_agent = initialize_agent(
    tools=[wikipedia_search, duck_search_eq, analizar_igualdad_web],
    llm=ollama_model,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
)


/tmp/ipykernel_2975019/1824066555.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [16]:
# 1) Pedido directo de investigación + análisis
q1 = "Investiga la cobertura sobre 'Irene Montero' (política) y evalúa si los titulares muestran sesgo de género. Resume y da acciones."

res1 = equality_agent.invoke({"input": q1})
print(res1["output"])




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: analizar_igualdad_web
Action Input: analizar_igualdad_web

/tmp/ipykernel_2975019/2395312617.py:18: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = duck_search_eq(query)
/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/tmp/ipykernel_2975019/2395312617.py:33: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resp = ollama_model(prompt)



Observation: ```json
{
  "resumen": "El texto proporcionado es un resultado negativo de la búsqueda utilizando DuckDuckGo. No contiene información sobre sesgos de género, ni resultados de búsqueda relevantes para esta tarea.  La tarea se centra en analizar resultados de búsqueda en busca de sesgos de género.  Por lo tanto, la tarea no ofrece evidencia para identificar ningún tipo de sesgo.",
  "etiqueta": "SIN_SEXISMO",
  "evidencias": [
    {
      "texto": "No good DuckDuckGo Search Result was found",
      "url": "URL_NO_DISPONIBLE",
      "porqué": "El texto en sí mismo es una declaración de que no se encontró un resultado de búsqueda, no proporciona información que pueda indicar sesgo de género. La ausencia de un resultado de búsqueda no implica la ausencia de sesgo, sólo que no fue encontrado."
    }
  ],
  "recomendaciones": [
    "Si el objetivo es detectar sesgos de género en los resultados de búsqueda, es crucial proporcionar a la herramienta de análisis palabras clave y fra

/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: ```json
{
  "1) resumen": "La definición de \"analizar\" según el RAE se centra en la acción de examinar o estudiar algo con cuidado. Se ejemplifica su uso en contextos como el análisis de problemas o obras literarias. El texto destaca la función tránsitoria del verbo, especificando que implica la realización de un análisis.",
  "2) etiqueta": "SIN_SEXISMO",
  "3) evidencias": [
    {
      "texto": "“El autor analiza los problemas económicos de las familias rurales”",
      "url": "https://www.rae.es/convlex/convlex_articulos_analizar",
      "porqué": "Aunque la frase es neutral en apariencia, la focalización en \"familias rurales\" podría, implícitamente, reforzar estereotipos de género asociados al ámbito doméstico y familiar, aunque no hay un sesgo explícito."
    },
    {
      "texto": "analizar una obra literaria",
      "url": "https://www.rae.es/convlex/convlex_articulos_analizar",
      "porqué": "El uso de 'obra literaria' como objeto de análisis no presenta s